In [1]:
import spacy
import fullmetalalchemy as fa
import pandas as pd
import sqlalchemy as sa

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [2]:
engine = fa.create.create_engine('sqlite:///data2.db')
text_table = fa.features.get_table('text', engine)
print(fa.features.get_row_count(text_table))
print(fa.features.get_column_names(text_table))
entity_table = fa.features.get_table('entities', engine)
print(fa.features.get_row_count(entity_table))
print(fa.features.get_column_names(entity_table))
found_table = fa.features.get_table('found_entities', engine)
print(fa.features.get_row_count(found_table))
print(fa.features.get_column_names(found_table))

In [ ]:
def insert_label(text_id: int, entity: str, label: str) -> None:
    # check if entity is in entity_table
    existing_entities = fa.select.select_records_by_primary_keys(entity_table, [{'entity': entity}])
    if existing_entities == []:
        # if not, insert into entity_table
        fa.insert.insert_records(entity_table, [{'entity': entity}])
        existing_entities = fa.select.select_records_by_primary_keys(entity_table, [{'entity': entity}])
    entity_id = existing_entities[0]['id']
    # add entity to found_entity table
    record = {'text_id': text_id, 'entity_id': entity_id, 'label': label}
    fa.insert.insert_records(found_table, [record, ])

In [ ]:
texts = fa.select.select_column_values_all(text_table, 'text')
ids = fa.select.select_column_values_all(text_table, 'id')
for text_id, doc in zip(ids, nlp.pipe(texts, n_process=-1)):
    for ent in doc.ents:
        insert_label(text_id, ent.text, ent.label_)

In [3]:
pd.read_sql_table('entities', engine)

In [4]:
pd.read_sql_table('found_entities', engine)